In [9]:
import datetime
import csv

from mwviews.api import PageviewsClient

# uses parallelism to for large calls
p = PageviewsClient(100)


# get current month
now = datetime.datetime.now()

date_string = str(now.year) + now.strftime('%m') + "0101"


def get_views(article_list):
    raw_views= p.article_views('en.wikipedia', article_list, granularity='monthly',
        start=date_string, end=date_string)
    for views in raw_views.itervalues():
        return views.items()

def append_views(views_list):
    """
    appends views data to csv
    """
    with open('some.csv', 'a') as f:
        writer = csv.writer(f)
        writer.writerows(views_list)

## Combine Available Files using Pandas

data:
https://dumps.wikimedia.org/other/pagecounts-raw/2016/2016-05/pagecounts-20160501-000000.gz

https://dumps.wikimedia.org/other/pagecounts-all-sites/2016/2016-01/

In [21]:
import pandas as pd
import requests
import zipfile, StringIO

In [20]:
zipped_views = requests.get("https://dumps.wikimedia.org/other/pagecounts-all-sites/2016/2016-05/pagecounts-20160501-000000.gz").content

In [28]:
import urllib

testfile = urllib.URLopener()
testfile.retrieve("https://dumps.wikimedia.org/other/pagecounts-all-sites/2016/2016-05/pagecounts-20160501-000000.gz",
                  "20160501-0.gz")

('20160501-0.gz', <httplib.HTTPMessage instance at 0x10fde75f0>)

In [30]:
! ls

20160501-0.gz             benchmark_streaming.ipynb get_first_link.pyc
__init__.py               build_reverse_fln.py      get_monthly_views.py
__init__.pyc              build_views_data.ipynb    some.csv
__pycache__               get_first_link.py


In [36]:
df = pd.read_csv("20160501-0.gz", compression="gzip", sep=" ", 
                 error_bad_lines=False, header=None)
df.columns = ['project', 'article', 'views', 'content_returned']

In [39]:
df.head()

,project,article,views,content_returned
0,aa,Category:Language_user_templates,1,6020
1,aa,File:Hitchhicking_on_the_road_132_-_Gasp%C3%A9...,1,8520
2,aa,File:Mortgage_Loan_graphs.jpg,1,8397
3,aa,File:Wikimedia_Community_Logo.svg,1,8993
4,aa,File:Www.wattpad.com/user/withdsufbasupufi_Loa...,1,4724


In [42]:
df.head()

,project,article,views,content_returned
0,aa,Category:Language_user_templates,1,6020
1,aa,File:Hitchhicking_on_the_road_132_-_Gasp%C3%A9...,1,8520
2,aa,File:Mortgage_Loan_graphs.jpg,1,8397
3,aa,File:Wikimedia_Community_Logo.svg,1,8993
4,aa,File:Www.wattpad.com/user/withdsufbasupufi_Loa...,1,4724


In [ ]:
def build_df(year, month, day, hour):
    """
    returns a pandas dataframe with columns:
        project, article, views, content_returned
    based on input date string:
        e.g, "2016", "06", "24", "01"
    files are found at
        https://dumps.wikimedia.org/other/pagecounts-all-sites/2016/2016-05/pagecounts-20160501-000000.gz
    """
    base_url_string = "https://dumps.wikimedia.org/other/pagecounts-all-sites/"
    url_string = base_url_string + "/" + year + "/" + year + "-" + month
    url_string += "/pagecounts-" + year + month + day + "-" + hour + "0000.gz"
    
    file_name = year + month + day + "-" + hour + ".gz"
    
    # download file
    opener = urllib.URLopener()    
    opener.retrieve(url_string, file_name)
    

    # return data frame
    df = pd.read_csv(file_name, compression="gzip", sep=" ", 
                 error_bad_lines=False, header=None)
    df.columns = ['project', 'article', 'views', 'content_returned']
    # include only en-wiki
    df = df[df['project'] == 'en']
    
    
    return df

In [44]:
df2 = build_df("2016", "06", "02", "01")

In [45]:
df2.head()

,project,article,views,content_returned
0,aa,File:Cash_payment_timeline_on_foreclosures.jpg,1,8505
1,aa,Main_Page,1,21579
2,aa,Main_Page/,1,14025
3,aa,User_talk:Alvaronicol%C3%A1s,1,4735
4,aa,User_talk:Andreas_S.,1,4729


In [ ]:
df.merge(df2, left_on='article', right_on='article', how='outer')